**HOMEWORK 3: Classifying Digits**

saru07g@uw.edu

**Goal:** train a classifier to distinguish images of handwritten digits from the famous MNIST data set

***Task 1*: You will need to reshape each image into a vector and stack the vectors into matrices 𝑋𝑡𝑟𝑎𝑖𝑛 and 𝑋𝑡𝑒𝑠𝑡
respectively. Perform PCA analysis of the digit images in the train set. Plot the first 16 PC modes as
28 × 28 images**

In [1]:
import struct 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

#load data 
filename = {'images' : 'train-images.idx3-ubyte' ,'labels' : 'train-labels.idx1-ubyte'}
filename2 = {'images_' : 't10k-images.idx3-ubyte', 'labels_' : 't10k-labels.idx1-ubyte'}

#load and parse
with open(filename['images'],'rb') as f:
    magic, size = struct.unpack(">II", f.read(8))
    nrows, ncols = struct.unpack(">II", f.read(8))
    data = np.fromfile(f, dtype=np.dtype(np.uint8).newbyteorder('>'))
    print(data.shape)
    Xtraindata = np.transpose(data.reshape((size, nrows*ncols))) #reshape
    
with open(filename['labels'],'rb') as f:
    magic, size = struct.unpack(">II", f.read(8))
    data = np.fromfile(f, dtype=np.dtype(np.uint8).newbyteorder('>'))
    ytrainlabels = data.reshape((size,))
    
with open(filename2['images_'],'rb') as f:
    magic, size = struct.unpack(">II", f.read(8))
    nrows, ncols = struct.unpack(">II", f.read(8))
    data = np.fromfile(f, dtype=np.dtype(np.uint8).newbyteorder('>'))
    Xtestdata = np.transpose(data.reshape((size, nrows*ncols))) #reshape

with open(filename2['labels_'],'rb') as f:
    magic, size = struct.unpack(">II", f.read(8))
    data = np.fromfile(f, dtype=np.dtype(np.uint8).newbyteorder('>'))
    ytestlabels = data.reshape((size,))
    
traindata_imgs =  np.transpose(Xtraindata).reshape((60000,28,28))    

X_train_vectors = Xtraindata.T

# Perform PCA on training set
pca = PCA(n_components=784)
X_train_pca = pca.fit_transform(X_train_vectors)

#plotting images
#def plot_pca_components(components, title):
    #fig, ax = plt.subplots(4, 4, figsize=(8, 8))
    
    #for i in range(4):
       #for j in range(4):
            #ax[i, j].imshow(components[4*i + j].reshape((28, 28)), cmap='gray')
           # ax[i, j].axis('off')
   # fig.suptitle(title, fontsize=24)

#plot_pca_components(pca.components_[:16], "First 16 Principal Components")


(47040000,)


***Task 2*: Inspect the cumulative energy of the singular values and determine 𝑘: the number of PC modes needed
to approximate 85% of the energy.**

In [2]:
singular_values = pca.singular_values_

# Calculate cumulative energy
cumulative_energy = np.cumsum(singular_values ** 2) / np.sum(singular_values ** 2)

# Determine k: number of PC modes needed to approximate 85% of the energy
k = np.argmax(cumulative_energy >= 0.85) + 1  # Add 1 because indexing starts from 0

#print("Number of PC modes needed to approximate 85% of the energy:", k)

# Plot cumulative energy
#plt.plot(cumulative_energy)
#plt.xlabel('Number of Principal Components')
#plt.ylabel('Cumulative Energy')
#plt.title('Cumulative Energy of Singular Values')
#plt.axvline(x=k, color='r', linestyle='--', label=f'{k} PC modes')
#plt.axhline(y=0.85, color='g', linestyle='--', label='85% energy')
#plt.legend()
#plt.show()

**You may also want to inspect several approximated digit images
reconstructed from 𝑘 truncated PC modes and plot them to make sure that the image reconstruction
using truncated modes is reasonable.**

In [3]:
def plot_reconstructed_images(images, title):
    fig, ax = plt.subplots(8, 8, figsize=(10, 10))
    
    for i in range(8):
        for j in range(8):
            ax[i, j].imshow(images[8*i + j].reshape((28, 28)), cmap='gray')
            ax[i, j].axis('off')
    fig.suptitle(title, fontsize=16)

pca = PCA(n_components=59)
X_train_pca = pca.fit_transform(X_train_vectors)

# Reconstruct images using the truncated PC modes
X_train_approx = pca.inverse_transform(X_train_pca)

# Plot first 64 reconstructed images in an 8x8 grid
#plot_reconstructed_images(X_train_approx[:64], "First 64 Reconstructed Images with k=59")
#plt.show()


***Task 3*: Write a function that selects a subset of particular digits (all samples of them) from 𝑋𝑡𝑟𝑎𝑖𝑛, 𝑦𝑡𝑟𝑎𝑖𝑛,
𝑋𝑡𝑒𝑠𝑡 and 𝑦𝑡𝑒𝑠𝑡 and returns the subset as new matrices 𝑋𝑠𝑢𝑏𝑡𝑟𝑎𝑖𝑛, 𝑦𝑠𝑢𝑏𝑡𝑟𝑎𝑖𝑛, 𝑋𝑠𝑢𝑏𝑡𝑒𝑠𝑡 and 𝑦𝑠𝑢𝑏𝑡𝑒𝑠𝑡.**

In [8]:
def select_digit_subset(digit, Xtraindata, ytrainlabels, Xtestdata, ytestlabels):
    train_indices = np.where(np.isin(ytrainlabels, digit))[0]
    test_indices = np.where(np.isin(ytestlabels, digit))[0]

    X_subtrain = Xtraindata[:, train_indices].T
    y_subtrain = ytrainlabels[train_indices]
    X_subtest = Xtestdata[:, test_indices].T
    y_subtest = ytestlabels[test_indices]

    return X_subtrain, y_subtrain, X_subtest, y_subtest

***Task 4*: Select the digits 1,8 using step 3, project the data onto 𝑘-PC modes computed in steps 1-2, and apply
the Ridge classifier (linear) to distinguish between these two digits. Perform cross-validation and
testing.**

In [9]:
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

selected_digits = [1, 8]
X_subtrain, y_subtrain, X_subtest, y_subtest = select_digit_subset(selected_digits, Xtraindata, ytrainlabels, Xtestdata, ytestlabels)

# Apply PCA on 59 components
pca = PCA(n_components=59)
pca_X_subtrain = pca.fit_transform(X_subtrain)
pca_X_subtest = pca.fit_transform(X_subtest)

# Ridge Classifier
ridge_clf = RidgeClassifier()


ridge_clf.fit(pca_X_subtrain, y_subtrain)

#print("Training Score: {}".format(ridge_clf.score(pca_X_subtrain, y_subtrain)))
#print("Testing Score: {}".format(ridge_clf.score(pca_X_subtest, y_subtest)))

cv_scores = cross_val_score(ridge_clf, pca_X_subtrain, y_subtrain, cv=5)
#print("{} accuracy with a standard deviation of {:0.5f}".format(cv_scores.mean(), cv_scores.std()))

***Task 5*: Repeat the same classification procedure for pairs of digits 3,8 and 2,7.**

In [ ]:
def classify_digit_pair(pair, Xtraindata, ytrainlabels, Xtestdata, ytestlabels):
    
    selected_digits = pair
    X_subtrain, y_subtrain, X_subtest, y_subtest = select_digit_subset(selected_digits, Xtraindata, ytrainlabels, Xtestdata, ytestlabels)
    
    # Apply PCA on 59 modes
    pca = PCA(n_components=59)
    pca_X_subtrain = pca.fit_transform(X_subtrain)
    pca_X_subtest = pca.transform(X_subtest)
    
    # Ridge Classifier
    ridge_clf = RidgeClassifier()
    ridge_clf.fit(pca_X_subtrain, y_subtrain)
    
    cv_scores = cross_val_score(ridge_clf, pca_X_subtrain, y_subtrain, cv=5)
    
    # Print
    #print("Training Score ({} , {}): {}".format(pair[0], pair[1], ridge_clf.score(pca_X_subtrain, y_subtrain)))
    #print("Testing Score ({} , {}): {}".format(pair[0], pair[1], ridge_clf.score(pca_X_subtest, y_subtest)))
    
   
    #print("Cross-validation accuracy ({} , {}): {} (std: {:0.5f})".format(pair[0], pair[1], cv_scores.mean(), cv_scores.std()))

# Classify pairs of digits: (3, 8) and (2, 7)
classify_digit_pair((3, 8), Xtraindata, ytrainlabels, Xtestdata, ytestlabels)
classify_digit_pair((2, 7), Xtraindata, ytrainlabels, Xtestdata, ytestlabels)



***Task 6*: Use all the digits and perform multi-class classification with Ridge, KNN and LDA classifiers.**

In [ ]:
from sklearn.linear_model import RidgeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

#classifiers
classifiers = {
    "Ridge Classifier": RidgeClassifier(),
    "Linear Discriminant Analysis": LinearDiscriminantAnalysis(),
    "K-Nearest Neighbors": KNeighborsClassifier()
}

# Apply PCA on 59 components
pca = PCA(n_components=59)
pca_X_train = pca.fit_transform(Xtraindata.T)
pca_X_test = pca.transform(Xtestdata.T)

# Define a range of values for n_neighbors
n_neighbors_values = [3, 4, 5]

# Loop over classifiers
for name, clf in classifiers.items():
    if name == "K-Nearest Neighbors":
        for n_neighbors in n_neighbors_values:
            # Apply StandardScaler for KNN classifier
            scaler = StandardScaler()
            pca_X_train_scaled = scaler.fit_transform(pca_X_train)
            pca_X_test_scaled = scaler.transform(pca_X_test)
            
            clf.set_params(n_neighbors=n_neighbors)
            clf.fit(pca_X_train_scaled, ytrainlabels)
            
            train_score = clf.score(pca_X_train_scaled, ytrainlabels)
            test_score = clf.score(pca_X_test_scaled, ytestlabels)
            
            #print(f"{name} (n_neighbors={n_neighbors}):")
            #print("  Training Score:", train_score)
            #print("  Testing Score:", test_score)
            # cross-validation score
            cv_scores = cross_val_score(clf, pca_X_train, ytrainlabels, cv=5)
            #print("  Cross-validation Score:", cv_scores.mean())
    else:
       
        clf.fit(pca_X_train, ytrainlabels)
        # Print
        train_score = clf.score(pca_X_train, ytrainlabels)
        test_score = clf.score(pca_X_test, ytestlabels)
        
        #print(f"{name}:")
        #print("  Training Score:", train_score)
        #print("  Testing Score:", test_score)

        # cross-validation score
        cv_scores = cross_val_score(clf, pca_X_train, ytrainlabels, cv=5)
        #print("  Cross-validation Score:", cv_scores.mean())


**confusion matrix for each classifier**

In [7]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

#fig, axes = plt.subplots(nrows=1, ncols=len(classifiers), figsize=(20, 5))

# Loop over classifiers
#for ax, (name, clf) in zip(axes, classifiers.items()):
    # predictions
    #y_pred = clf.predict(pca_X_test)

    # Plot confusion matrix
    #cm_display = ConfusionMatrixDisplay.from_predictions(y_pred, ytestlabels, ax=ax)

    # Set class names for x-axis and y-axis
    #classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    #cm_display.ax_.set_xticklabels(classes)
    #cm_display.ax_.set_yticklabels(classes)

    #ax.set_title(f"Confusion Matrix for {name}")

#plt.tight_layout()
#plt.show()
